In [1]:
import pickle
from pathlib import Path
import numpy as np
import pandas as ps
import category_encoders as ce
import datetime
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sbn
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

ps.set_option('display.max_rows', 500)
ps.set_option('display.max_columns', 500)
ps.set_option('display.width', 1000)

In [2]:
data_folder = Path('..') / 'data'
test_df = ps.read_csv(data_folder / 'test.csv', encoding='latin-1')
test_df['CancelFlag'] = 0
train_df = ps.read_csv(data_folder / 'train.csv', encoding='latin-1')

print('Train shapes:', train_df.shape)
print('Test shapes:', test_df.shape)

Train shapes: (9023184, 14)
Test shapes: (5032740, 14)


In [3]:
train_df[train_df['CancelFlag'] == 1].head()

,Interval,Date,OrderDate,ClientID,ChannelID,OrderID,MaterialID,GroupID,Cluster,CancelFlag,OrderCnt,DeliveryType,prepay,count_edit
28,10-18.,17/10/2018,15/10/2018,93411902,2,90102104012,3294844.0,36.0,NaN,1,1.0,Îáû÷íàÿ äîñòàâêà,0,1
29,10-18.,17/10/2018,15/10/2018,93411902,2,90102104012,3309718.0,30.0,NaN,1,1.0,Îáû÷íàÿ äîñòàâêà,0,1
30,10-18.,17/10/2018,15/10/2018,93411902,2,90102104012,3043258.0,63.0,NaN,1,2.0,Îáû÷íàÿ äîñòàâêà,0,1
31,10-18.,17/10/2018,15/10/2018,93411902,2,90102104012,3040464.0,17.0,NaN,1,1.0,Îáû÷íàÿ äîñòàâêà,0,1
32,10-18.,17/10/2018,15/10/2018,93411902,2,90102104012,3349229.0,12.0,NaN,1,1.0,Îáû÷íàÿ äîñòàâêà,0,1


In [4]:
# train_df[train_df['ClientID'] == 203679]

In [5]:
holidays_set = {
    '01/01/2018', '02/01/2018', '03/01/2018',
    '04/01/2018', '05/01/2018', '06/01/2018',
    '07/01/2018', '23/02/2018', '08/03/2018',
    '09/03/2018', '30/04/2018', '01/05/2018',
    '09/05/2018', '11/06/2018', '12/06/2018',
    '04/11/2018', '05/11/2018', '25/11/2018',
}

def is_holiday(d: str) -> bool:
    return d in holidays_set

In [6]:
def map_all_the_stuff(d: ps.DataFrame) -> ps.DataFrame:
    # mappings
    d['DeliveryType'] = d['DeliveryType'].map({'Îáû÷íàÿ äîñòàâêà': 0, 'Äîñòàâêà Äåíü â Äåíü': 1})
    d['Cluster'] = d['Cluster'].fillna('MISSING')
    d['OrderCnt'] = d['OrderCnt'].fillna(0)
    d['GroupID'] = d['GroupID'].fillna(0)
    d['MaterialID'] = d['MaterialID'].fillna(0)
    
    # intervals
    d['StartInterval'] = d['Interval'].apply(lambda item: int(item.rsplit('-')[0]))
    d['EndInterval'] = d['Interval'].apply(lambda item: int(item.rsplit('-')[1][:-1]))
    
    # dates
    d['Date_is_holiday'] = d['Date'].apply(is_holiday)
    d['Date'] = ps.to_datetime(d['Date'], format='%d/%m/%Y')
#     d['Date_day'] = d['Date'].dt.day
#     d['Date_month'] = d['Date'].dt.month
#     d['Date_year'] = d['Date'].dt.year
    d['Date_weekday'] = d['Date'].dt.weekday
    
    d['OrderDate_is_holiday'] = d['OrderDate'].apply(is_holiday)
    d['OrderDate'] = ps.to_datetime(d['OrderDate'], format='%d/%m/%Y')
#     d['OrderDate_day'] = d['OrderDate'].dt.day
#     d['OrderDate_month'] = d['OrderDate'].dt.month
#     d['OrderDate_year'] = d['OrderDate'].dt.year
    d['OrderDate_weekday'] = d['OrderDate'].dt.weekday
    
    d['DatesGap'] = d['Date'] - d['OrderDate']
    d['DatesGap_days'] = d['DatesGap'].dt.days
    
    # cast types
    d['MaterialID'] = d['MaterialID'].astype(np.int32)
    d['GroupID'] = d['GroupID'].astype(np.int16)
    d['OrderCnt'] = d['OrderCnt'].astype(np.int16)
    
#     d.drop(columns=['Interval', 'Date', 'OrderDate', 'DatesGap'], inplace=True)
    return d


def client_orders_cnt(d: ps.DataFrame) -> ps.DataFrame:
    client_order_cnt = d.groupby(['ClientID']).agg({'OrderID': ps.Series.nunique}).reset_index()
    client_order_cnt = client_order_cnt.rename(columns={'OrderID': 'ClientOrderCnt'})
    client_order_cnt['HasPreviousOrder'] = client_order_cnt['ClientOrderCnt'] > 1
    d = d.merge(client_order_cnt, on='ClientID', how='left')
    return d


def order_days_delay(d: ps.DataFrame) -> ps.DataFrame:
    order_dates = d.groupby(['ClientID', 'OrderID']).agg({'OrderDate': 'first'}).reset_index()
    order_dates['ShiftedOrderDate'] = order_dates.groupby('ClientID')['OrderDate'].shift(1)
    order_dates['OrderDatesDaysDelay'] = (order_dates['OrderDate'] - order_dates['ShiftedOrderDate']).dt.days
    order_dates['OrderDatesDaysDelay'] = order_dates['OrderDatesDaysDelay'].fillna(0).astype(int)
    order_dates = order_dates[['ClientID', 'OrderID', 'OrderDatesDaysDelay']]
    d = d.merge(order_dates, on=['ClientID', 'OrderID'], how='left')
    return d

In [7]:
%%time

train_df = map_all_the_stuff(train_df)
test_df = map_all_the_stuff(test_df)

CPU times: user 1min 11s, sys: 1.6 s, total: 1min 13s
Wall time: 50.4 s


In [8]:
%%time

train_df = client_orders_cnt(train_df)
test_df = client_orders_cnt(test_df)

CPU times: user 13.8 s, sys: 2.76 s, total: 16.5 s
Wall time: 8.8 s


In [9]:
%%time

train_df = order_days_delay(train_df)
test_df = order_days_delay(test_df)

CPU times: user 13.5 s, sys: 3.07 s, total: 16.6 s
Wall time: 6.05 s


In [10]:
train_df.drop(columns=['Interval', 'Date', 'OrderDate', 'DatesGap'], inplace=True)
test_df.drop(columns=['Interval', 'Date', 'OrderDate', 'DatesGap'], inplace=True)

In [11]:
train_df.head()

,ClientID,ChannelID,OrderID,MaterialID,GroupID,Cluster,CancelFlag,OrderCnt,DeliveryType,prepay,count_edit,StartInterval,EndInterval,Date_is_holiday,Date_weekday,OrderDate_is_holiday,OrderDate_weekday,DatesGap_days,ClientOrderCnt,HasPreviousOrder,OrderDatesDaysDelay
0,93808186,2,90102063002,3328810,61,MISSING,0,1,0,0,1,14,16,False,2,False,1,1,35,True,0
1,93808186,2,90102063002,3281258,30,MISSING,0,2,0,0,1,14,16,False,2,False,1,1,35,True,0
2,93808186,2,90102063002,3210734,10,MISSING,0,1,0,0,1,14,16,False,2,False,1,1,35,True,0
3,93808186,2,90102063002,3328848,61,MISSING,0,2,0,0,1,14,16,False,2,False,1,1,35,True,0
4,94112406,2,90102091007,3347801,17,MISSING,0,10,0,0,1,12,14,False,6,False,4,2,7,True,0


In [12]:
# %%time
# cat_cols = ['Cluster']

# for c in cat_cols:
#     le = LabelEncoder()
#     le.fit(np.concatenate([train_df[c], test_df[c]]))
#     train_df[c] = le.transform(train_df[c])
#     test_df[c] = le.transform(test_df[c])

In [13]:
%%time
cols2ohe = ['ChannelID', 'Cluster', 'GroupID']
one_hot_encoder = ce.OneHotEncoder(verbose=1, cols=cols2ohe)

one_hot_encoder.fit(ps.concat([train_df, test_df]))

CPU times: user 45.9 s, sys: 25.8 s, total: 1min 11s
Wall time: 29.3 s


OneHotEncoder(cols=['ChannelID', 'Cluster', 'GroupID'], drop_invariant=False,
              handle_missing='value', handle_unknown='value', return_df=True,
              use_cat_names=False, verbose=1)

In [14]:
%%time
train_df = one_hot_encoder.transform(train_df)

CPU times: user 25.4 s, sys: 14.4 s, total: 39.9 s
Wall time: 15.6 s


In [15]:
train_df.head()

,ClientID,ChannelID_1,ChannelID_2,ChannelID_3,ChannelID_4,ChannelID_5,OrderID,MaterialID,GroupID_1,GroupID_2,GroupID_3,GroupID_4,GroupID_5,GroupID_6,GroupID_7,GroupID_8,GroupID_9,GroupID_10,GroupID_11,GroupID_12,GroupID_13,GroupID_14,GroupID_15,GroupID_16,GroupID_17,GroupID_18,GroupID_19,GroupID_20,GroupID_21,GroupID_22,GroupID_23,GroupID_24,GroupID_25,GroupID_26,GroupID_27,GroupID_28,GroupID_29,GroupID_30,GroupID_31,GroupID_32,GroupID_33,GroupID_34,GroupID_35,GroupID_36,GroupID_37,GroupID_38,GroupID_39,GroupID_40,GroupID_41,GroupID_42,GroupID_43,GroupID_44,GroupID_45,GroupID_46,GroupID_47,GroupID_48,GroupID_49,Cluster_1,Cluster_2,Cluster_3,Cluster_4,Cluster_5,Cluster_6,Cluster_7,Cluster_8,Cluster_9,Cluster_10,Cluster_11,Cluster_12,Cluster_13,Cluster_14,Cluster_15,Cluster_16,Cluster_17,Cluster_18,Cluster_19,Cluster_20,Cluster_21,Cluster_22,Cluster_23,Cluster_24,Cluster_25,Cluster_26,Cluster_27,Cluster_28,Cluster_29,CancelFlag,OrderCnt,DeliveryType,prepay,count_edit,StartInterval,EndInterval,Date_is_holiday,Date_weekday,OrderDate_is_holiday,OrderDate_weekday,DatesGap_days,ClientOrderCnt,HasPreviousOrder,OrderDatesDaysDelay
0,93808186,1,0,0,0,0,90102063002,3328810,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,14,16,False,2,False,1,1,35,True,0
1,93808186,1,0,0,0,0,90102063002,3281258,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,14,16,False,2,False,1,1,35,True,0
2,93808186,1,0,0,0,0,90102063002,3210734,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,14,16,False,2,False,1,1,35,True,0
3,93808186,1,0,0,0,0,90102063002,3328848,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,14,16,False,2,False,1,1,35,True,0
4,94112406,1,0,0,0,0,90102091007,3347801,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,1,12,14,False,6,False,4,2,7,True,0


In [16]:
%%time

test_df = one_hot_encoder.transform(test_df)

CPU times: user 17.4 s, sys: 7.96 s, total: 25.3 s
Wall time: 8.76 s


In [17]:
test_df.head(10)

,ClientID,ChannelID_1,ChannelID_2,ChannelID_3,ChannelID_4,ChannelID_5,OrderID,MaterialID,GroupID_1,GroupID_2,GroupID_3,GroupID_4,GroupID_5,GroupID_6,GroupID_7,GroupID_8,GroupID_9,GroupID_10,GroupID_11,GroupID_12,GroupID_13,GroupID_14,GroupID_15,GroupID_16,GroupID_17,GroupID_18,GroupID_19,GroupID_20,GroupID_21,GroupID_22,GroupID_23,GroupID_24,GroupID_25,GroupID_26,GroupID_27,GroupID_28,GroupID_29,GroupID_30,GroupID_31,GroupID_32,GroupID_33,GroupID_34,GroupID_35,GroupID_36,GroupID_37,GroupID_38,GroupID_39,GroupID_40,GroupID_41,GroupID_42,GroupID_43,GroupID_44,GroupID_45,GroupID_46,GroupID_47,GroupID_48,GroupID_49,Cluster_1,Cluster_2,Cluster_3,Cluster_4,Cluster_5,Cluster_6,Cluster_7,Cluster_8,Cluster_9,Cluster_10,Cluster_11,Cluster_12,Cluster_13,Cluster_14,Cluster_15,Cluster_16,Cluster_17,Cluster_18,Cluster_19,Cluster_20,Cluster_21,Cluster_22,Cluster_23,Cluster_24,Cluster_25,Cluster_26,Cluster_27,Cluster_28,Cluster_29,OrderCnt,DeliveryType,prepay,count_edit,CancelFlag,StartInterval,EndInterval,Date_is_holiday,Date_weekday,OrderDate_is_holiday,OrderDate_weekday,DatesGap_days,ClientOrderCnt,HasPreviousOrder,OrderDatesDaysDelay
0,93307117,1,0,0,0,0,90102211131,3056480,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,21,22,False,4,False,4,0,26,True,0
1,93307117,1,0,0,0,0,90102211131,2014233,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,1,0,21,22,False,4,False,4,0,26,True,0
2,93307117,1,0,0,0,0,90102211131,3226516,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,21,22,False,4,False,4,0,26,True,0
3,91590087,1,0,0,0,0,90102211133,2013713,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,22,0,False,5,False,4,1,12,True,0
4,91590087,1,0,0,0,0,90102211133,3255780,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,22,0,False,5,False,4,1,12,True,0
5,91590087,1,0,0,0,0,90102211133,2014036,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,22,0,False,5,False,4,1,12,True,0
6,91590087,1,0,0,0,0,90102211133,3157954,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,22,0,False,5,False,4,1,12,True,0
7,91590087,1,0,0,0,0,90102211133,3357487,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,1,0,22,0,False,5,False,4,1,12,True,0
8,91590087,1,0,0,0,0,90102211133,3258369,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,22,0,False,5,False,4,1,12,True,0
9,91590087,1,0,0,0,0,90102211133,3347895,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,22,0,False,5,False,4,1,12,True,0


In [18]:
train_df['MaterialsCnt'] = 1
test_df['MaterialsCnt'] = 1

In [19]:
train_df = train_df.drop(columns=['ClientID', 'MaterialID'])
train_df = train_df.groupby('OrderID').agg({
    c: 'sum' if c.startswith('ChannelID') or \
    c.startswith('GroupID') or \
    c.startswith('Cluster') or \
    c.startswith('OrderCnt') or \
    c.startswith('MaterialsCnt') \
    else 'mean'
    for c in train_df.columns
}).drop(columns=['OrderID']).reset_index()

print(train_df.shape)
train_df.head()

(354851, 100)


,OrderID,ChannelID_1,ChannelID_2,ChannelID_3,ChannelID_4,ChannelID_5,GroupID_1,GroupID_2,GroupID_3,GroupID_4,GroupID_5,GroupID_6,GroupID_7,GroupID_8,GroupID_9,GroupID_10,GroupID_11,GroupID_12,GroupID_13,GroupID_14,GroupID_15,GroupID_16,GroupID_17,GroupID_18,GroupID_19,GroupID_20,GroupID_21,GroupID_22,GroupID_23,GroupID_24,GroupID_25,GroupID_26,GroupID_27,GroupID_28,GroupID_29,GroupID_30,GroupID_31,GroupID_32,GroupID_33,GroupID_34,GroupID_35,GroupID_36,GroupID_37,GroupID_38,GroupID_39,GroupID_40,GroupID_41,GroupID_42,GroupID_43,GroupID_44,GroupID_45,GroupID_46,GroupID_47,GroupID_48,GroupID_49,Cluster_1,Cluster_2,Cluster_3,Cluster_4,Cluster_5,Cluster_6,Cluster_7,Cluster_8,Cluster_9,Cluster_10,Cluster_11,Cluster_12,Cluster_13,Cluster_14,Cluster_15,Cluster_16,Cluster_17,Cluster_18,Cluster_19,Cluster_20,Cluster_21,Cluster_22,Cluster_23,Cluster_24,Cluster_25,Cluster_26,Cluster_27,Cluster_28,Cluster_29,CancelFlag,OrderCnt,DeliveryType,prepay,count_edit,StartInterval,EndInterval,Date_is_holiday,Date_weekday,OrderDate_is_holiday,OrderDate_weekday,DatesGap_days,ClientOrderCnt,HasPreviousOrder,OrderDatesDaysDelay,MaterialsCnt
0,90102063002,4,0,0,0,0,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,1,14,16,False,2,False,1,1,35,True,0,4
1,90102091007,5,0,0,0,0,0,0,0,2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28,0,0,1,12,14,False,6,False,4,2,7,True,0,5
2,90102092000,10,0,0,0,0,0,0,0,0,2,0,0,1,3,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,1,0,1,19,21,False,4,False,4,0,36,True,0,10
3,90102103017,9,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,4,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,0,0,1,22,0,False,4,False,4,0,36,True,7,9
4,90102104012,26,0,0,0,0,1,3,0,2,0,1,0,0,0,0,0,2,0,0,0,11,2,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,29,0,0,1,10,18,False,2,False,0,2,18,True,0,26


In [20]:
test_df = test_df.drop(columns=['ClientID', 'MaterialID', 'CancelFlag'])
test_df = test_df.groupby('OrderID').agg({
    c: 'sum' if c.startswith('ChannelID') or \
    c.startswith('GroupID') or \
    c.startswith('Cluster') or \
    c.startswith('OrderCnt') or \
    c.startswith('MaterialsCnt') \
    else 'mean'
    for c in test_df.columns
}).drop(columns=['OrderID']).reset_index()

print(test_df.shape)
test_df.head()

(202614, 99)


,OrderID,ChannelID_1,ChannelID_2,ChannelID_3,ChannelID_4,ChannelID_5,GroupID_1,GroupID_2,GroupID_3,GroupID_4,GroupID_5,GroupID_6,GroupID_7,GroupID_8,GroupID_9,GroupID_10,GroupID_11,GroupID_12,GroupID_13,GroupID_14,GroupID_15,GroupID_16,GroupID_17,GroupID_18,GroupID_19,GroupID_20,GroupID_21,GroupID_22,GroupID_23,GroupID_24,GroupID_25,GroupID_26,GroupID_27,GroupID_28,GroupID_29,GroupID_30,GroupID_31,GroupID_32,GroupID_33,GroupID_34,GroupID_35,GroupID_36,GroupID_37,GroupID_38,GroupID_39,GroupID_40,GroupID_41,GroupID_42,GroupID_43,GroupID_44,GroupID_45,GroupID_46,GroupID_47,GroupID_48,GroupID_49,Cluster_1,Cluster_2,Cluster_3,Cluster_4,Cluster_5,Cluster_6,Cluster_7,Cluster_8,Cluster_9,Cluster_10,Cluster_11,Cluster_12,Cluster_13,Cluster_14,Cluster_15,Cluster_16,Cluster_17,Cluster_18,Cluster_19,Cluster_20,Cluster_21,Cluster_22,Cluster_23,Cluster_24,Cluster_25,Cluster_26,Cluster_27,Cluster_28,Cluster_29,OrderCnt,DeliveryType,prepay,count_edit,StartInterval,EndInterval,Date_is_holiday,Date_weekday,OrderDate_is_holiday,OrderDate_weekday,DatesGap_days,ClientOrderCnt,HasPreviousOrder,OrderDatesDaysDelay,MaterialsCnt
0,90102211131,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1,0,1,21,22,False,4,False,4,0,26,True,0,3
1,90102211133,23,0,0,0,0,2,1,0,1,0,0,0,0,0,2,0,1,0,0,0,10,0,0,1,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,44,0,0,1,22,0,False,5,False,4,1,12,True,0,23
2,90102216055,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,1,18,20,False,3,False,2,1,24,True,0,1
3,90102216081,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,1,6,14,False,4,False,3,1,24,True,1,1
4,90102216084,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,1,10,14,False,5,False,3,2,5,True,0,6


In [21]:
with open(str(data_folder / 'processed' / 'train_3.pkl'), 'wb') as f:
    pickle.dump(train_df, f)
# train_df.to_csv(data_folder / 'processed' / 'train.csv', index=False)

In [22]:
with open(str(data_folder / 'processed' / 'test_3.pkl'), 'wb') as f:
    pickle.dump(test_df, f)
# test_df.to_csv(data_folder / 'processed' / 'test.csv', index=False)

In [23]:
train_df['CancelFlag'].value_counts()

0    331075
1     23776
Name: CancelFlag, dtype: int64

In [19]:
train_df.columns.tolist()

['OrderID',
 'ChannelID_1',
 'ChannelID_2',
 'ChannelID_3',
 'ChannelID_4',
 'ChannelID_5',
 'GroupID_1',
 'GroupID_2',
 'GroupID_3',
 'GroupID_4',
 'GroupID_5',
 'GroupID_6',
 'GroupID_7',
 'GroupID_8',
 'GroupID_9',
 'GroupID_10',
 'GroupID_11',
 'GroupID_12',
 'GroupID_13',
 'GroupID_14',
 'GroupID_15',
 'GroupID_16',
 'GroupID_17',
 'GroupID_18',
 'GroupID_19',
 'GroupID_20',
 'GroupID_21',
 'GroupID_22',
 'GroupID_23',
 'GroupID_24',
 'GroupID_25',
 'GroupID_26',
 'GroupID_27',
 'GroupID_28',
 'GroupID_29',
 'GroupID_30',
 'GroupID_31',
 'GroupID_32',
 'GroupID_33',
 'GroupID_34',
 'GroupID_35',
 'GroupID_36',
 'GroupID_37',
 'GroupID_38',
 'GroupID_39',
 'GroupID_40',
 'GroupID_41',
 'GroupID_42',
 'GroupID_43',
 'GroupID_44',
 'GroupID_45',
 'GroupID_46',
 'GroupID_47',
 'GroupID_48',
 'GroupID_49',
 'Cluster_1',
 'Cluster_2',
 'Cluster_3',
 'Cluster_4',
 'Cluster_5',
 'Cluster_6',
 'Cluster_7',
 'Cluster_8',
 'Cluster_9',
 'Cluster_10',
 'Cluster_11',
 'Cluster_12',
 'Cluster_1